<a href="https://colab.research.google.com/github/HatemMoushir/Shark-identification-1/blob/main/Shark%20identification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

path = "/content/drive/MyDrive/Shark_project"
files = os.listdir(path)
print(files)

['shark_fins_raw', 'sharks_raw.zip']


In [ ]:
!pip install -q imagehash Pillow pytesseract

import os
from PIL import Image, UnidentifiedImageError
import imagehash
import pytesseract
from collections import defaultdict
import matplotlib.pyplot as plt

base_path = "/content/drive/MyDrive/Shark_project"
min_size = (100, 100)
hash_dict = defaultdict(list)
duplicates = defaultdict(list)
bad_images = []
small_images = []
images_with_text = []

def display_images(image_paths, title):
    print(f"\n📸 {title} - عدد الصور: {len(image_paths)}\n")
    for path in image_paths[:10]:
        try:
            img = Image.open(path).convert("RGB")
            plt.imshow(img)
            plt.title(os.path.basename(path))
            plt.axis('off')
            plt.show()
        except Exception as e:
            print(f"❌ خطأ أثناء العرض: {path}", e)

def has_text(img):
    text = pytesseract.image_to_string(img)
    return bool(text.strip())

for root, dirs, files in os.walk(base_path):
    for img_name in files:
        img_path = os.path.join(root, img_name)
        if not os.path.isfile(img_path):
            continue

        try:
            img = Image.open(img_path)
            img.verify()
            img = Image.open(img_path).convert("RGB")

            if img.size[0] < min_size[0] or img.size[1] < min_size[1]:
                small_images.append(img_path)

            hash_val = imagehash.phash(img)
            hash_dict[str(hash_val)].append(img_path)

            if has_text(img):
                images_with_text.append(img_path)

        except (UnidentifiedImageError, OSError) as e:
            bad_images.append(img_path)

for h, paths in hash_dict.items():
    if len(paths) > 1:
        duplicates[h] = paths

if small_images:
    display_images(small_images, "صور صغيرة الحجم")

if images_with_text:
    display_images(images_with_text, "صور تحتوي على كتابة / علامة مائية")

if duplicates:
    print(f"\n🔁 مجموعات صور مكررة بصريًا: {len(duplicates)}\n")
    for i, (h, paths) in enumerate(duplicates.items()):
        display_images(paths, f"صور مكررة - مجموعة {i+1}")

if bad_images:
    display_images(bad_images, "صور تالفة")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.4 MB/s eta 0:00:00


In [11]:
dataset = ImageFolder("/content/Shark_project_split/train")
print(dataset.classes)

['Blacktip_Shark', 'Bull_Shark', 'Great_White_Shark', 'Hammerhead_Shark', 'Mako_Shark', 'Tiger_Shark', 'Whale_Shark', 'Whitetip_Shark']


In [ ]:

# تثبيت المكتبات المطلوبة
!pip install -q imagehash Pillow

import os
from PIL import Image, UnidentifiedImageError
import imagehash
from collections import defaultdict
import matplotlib.pyplot as plt

# مسار المشروع في Google Drive
#base_path = "/content/drive/MyDrive/Shark_project"
base_path = "/content/Shark_project"

# إعدادات عامة
min_size = (100, 100)
hash_dict = defaultdict(list)
duplicates = defaultdict(list)
bad_images = []
small_images = []

# دالة عرض الصور
def display_images(image_paths, title):
    print(f"\n📸 {title} - عدد الصور: {len(image_paths)}\n")
    for i, path in enumerate(image_paths[:10]):  # نعرض فقط أول 10 صور
        try:
            img = Image.open(path).convert("RGB")
            plt.imshow(img)
            plt.title(f"{os.path.basename(path)}")
            plt.axis('off')
            plt.show()
        except Exception as e:
            print(f"❌ خطأ أثناء العرض: {path}", e)

# -------------------------------
# فحص شامل لكل الصور بشكل Recursive
# -------------------------------
for root, dirs, files in os.walk(base_path):
    for img_name in files:
        img_path = os.path.join(root, img_name)

        if not os.path.isfile(img_path):
            continue

        try:
            img = Image.open(img_path)
            img.verify()  # فحص إذا كانت تالفة
            img = Image.open(img_path).convert("RGB")  # إعادة الفتح لتحليلها

            # فحص الحجم
            if img.size[0] < min_size[0] or img.size[1] < min_size[1]:
                small_images.append(img_path)

            # حساب hash
            hash_val = imagehash.phash(img)
            hash_dict[str(hash_val)].append(img_path)

        except (UnidentifiedImageError, OSError) as e:
            bad_images.append(img_path)

# -------------------------------
# معالجة الصور المكررة بناءً على phash
# -------------------------------
for h, paths in hash_dict.items():
    if len(paths) > 1:
        duplicates[h] = paths

# -------------------------------
# عرض النتائج
# -------------------------------

# الصور صغيرة الحجم
if small_images:
    display_images(small_images, "صور صغيرة الحجم (< 100x100)")

# الصور المكررة بصريًا
if duplicates:
    print(f"\n🔁 مجموعات صور مكررة بصريًا: {len(duplicates)}\n")
    for i, (h, paths) in enumerate(duplicates.items()):
        print(f"\n🧩 المجموعة {i+1} - Hash: {h} - عدد الصور: {len(paths)}")
        display_images(paths, f"صور مكررة - مجموعة {i+1}")

# الصور التالفة
if bad_images:
    display_images(bad_images, "صور تالفة")

In [ ]:
# -------------------------------
# حذف الصور المكررة (نترك واحدة فقط)
# -------------------------------

deleted_count = 0
for h, paths in duplicates.items():
    # نترك أول صورة، ونحذف الباقي
    for duplicate_path in paths[1:]:
        try:
            os.remove(duplicate_path)
            deleted_count += 1
            print(f"🗑️ حذف: {duplicate_path}")
        except Exception as e:
            print(f"❌ فشل في الحذف: {duplicate_path} - {e}")

print(f"\n✅ تم حذف {deleted_count} صورة مكررة وترك نسخة واحدة فقط من كل مجموعة.")

In [ ]:
from collections import Counter

class_counts = Counter()

for root, dirs, files in os.walk("/content/Shark_project"):
    for d in dirs:
        folder = os.path.join(root, d)
        count = len([f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
        class_counts[d] = count

print("📊 توزيع الصور على الفئات:")
for class_name, count in class_counts.items():
    print(f"{class_name:25s}: {count} صورة")

In [ ]:

import os

# المسار الجديد بناءً على كلامك
base_dir = "/content/Shark_project/shark_fins_raw"

# خريطة الأسماء التي نريد توحيدها
rename_map = {
    "Bull shark": "Bull_Shark",
    "Mako shark": "Mako_Shark",
    "Tiger_shark": "Tiger_Shark",
    "Great_White_Shark": "Great_White_Shark",
    "Blacktip_Shark": "Blacktip_Shark",
    "Hammerhead_Shark": "Hammerhead_Shark",
    "Whale_Shark": "Whale_Shark",
    "Whitetip_Shark": "Whitetip_Shark",
}

# تنفيذ إعادة التسمية
for old_name, new_name in rename_map.items():
    old_path = os.path.join(base_dir, old_name)
    new_path = os.path.join(base_dir, new_name)
    if os.path.exists(old_path):
        os.rename(old_path, new_path)
        print(f"✅ Renamed: {old_name} → {new_name}")
    else:
        print(f"⚠️ Skipped (not found): {old_name}")

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# مجلد الصور
base_dir = "/content/Shark_project/shark_fins_raw"

# الحصول على أسماء الأصناف
shark_classes = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])

# لكل صنف، نعرض 3 صور
for shark_class in shark_classes:
    class_path = os.path.join(base_dir, shark_class)
    images = [img for img in os.listdir(class_path) if img.lower().endswith((".jpg", ".jpeg", ".png"))]

    # لو مفيش صور، نعدي
    if len(images) == 0:
        continue

    # نختار 3 صور عشوائيًا (أو أقل لو العدد مش كافي)
    sample_images = random.sample(images, min(3, len(images)))

    print(f"\n🦈 {shark_class} — Showing {len(sample_images)} image(s):")

    # عرض الصور
    plt.figure(figsize=(12, 4))
    for i, img_name in enumerate(sample_images):
        img_path = os.path.join(class_path, img_name)
        img = mpimg.imread(img_path)
        plt.subplot(1, len(sample_images), i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{shark_class}")
    plt.show()

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# المجلد الأساسي للصور قبل التقسيم
source_dir = "/content/Shark_project/shark_fins_raw"
# المجلد الذي سننشئ فيه المجلدات المقسمة
dest_dir = "/content/Shark_project_split"

# النسب
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# إنشاء مجلدات train, val, test
splits = ['train', 'val', 'test']
for split in splits:
    split_path = os.path.join(dest_dir, split)
    os.makedirs(split_path, exist_ok=True)

# لكل صنف ننسخ الصور حسب النسبة
for class_name in tqdm(os.listdir(source_dir)):
    class_path = os.path.join(source_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    n_total = len(images)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)

    split_counts = {
        'train': images[:n_train],
        'val': images[n_train:n_train+n_val],
        'test': images[n_train+n_val:]
    }

    for split in splits:
        split_class_dir = os.path.join(dest_dir, split, class_name)
        os.makedirs(split_class_dir, exist_ok=True)
        for img_file in split_counts[split]:
            src_path = os.path.join(class_path, img_file)
            dst_path = os.path.join(split_class_dir, img_file)
            shutil.copy2(src_path, dst_path)

100%|██████████| 8/8 [00:00<00:00, 18.78it/s]


In [ ]:
!unzip "/content/sharks_raw.zip "-d "/content/Shark_project"

In [ ]:

# هذا هو الـ File ID من الرابط:
# https://drive.google.com/file/d/14uDazr8KcJqzV4A52EuQ-gOMoorWEOek/view
# ID هو: 14uDazr8KcJqzV4A52EuQ-gOMoorWEOek

!gdown 14uDazr8KcJqzV4A52EuQ-gOMoorWEOek

Downloading...
From (original): https://drive.google.com/uc?id=14uDazr8KcJqzV4A52EuQ-gOMoorWEOek
From (redirected): https://drive.google.com/uc?id=14uDazr8KcJqzV4A52EuQ-gOMoorWEOek&confirm=t&uuid=0515d6eb-233a-4775-81da-ed0b1109cf1f
To: /content/sharks_raw.zip
100% 142M/142M [00:01<00:00, 83.0MB/s]


In [ ]:
!unzip "/content/sharks_raw.zip" -d "/content/Shark_project"

In [ ]:
pip install wandb

In [ ]:
!pip install -q transformers datasets torchvision evaluate
!pip install wandb


from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import os

# المسار إلى البيانات
data_dir = "/content/Shark_project_split"

# التحويلات المبدئية (تصغير الصور وتحويلها إلى Tensor)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# تحميل البيانات
dataset = ImageFolder(data_dir, transform=transform)

# تقسيم إلى تدريب واختبار
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


#---

from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer, ViTImageProcessor
import torch
import numpy as np
from datasets import Dataset as HFDataset

# استخلاص الخصائص
# feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224") # Deprecated
feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

# تعريف نموذج التصنيف
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(dataset.classes),
    id2label={str(i): c for i, c in enumerate(dataset.classes)},
    label2id={c: str(i) for i, c in enumerate(dataset.classes)},
    ignore_mismatched_sizes=True # Add this argument to ignore the size mismatch in the classifier layer
)

#----

from PIL import Image

def transform_example(example):
    image = example['image']
    encoding = feature_extractor(images=image, return_tensors="pt")
    encoding['label'] = example['label']
    return encoding

# تحويل بيانات PyTorch إلى Dataset من نوع Hugging Face
def convert_to_hf_dataset(torch_dataset):
    images = []
    labels = []
    for img, label in torch_dataset:
        images.append(img.permute(1, 2, 0).numpy())  # Convert to HWC
        labels.append(label)
    return HFDataset.from_dict({"image": images, "label": labels})

hf_train = convert_to_hf_dataset(train_dataset).with_transform(transform_example)
hf_val = convert_to_hf_dataset(val_dataset).with_transform(transform_example)


#----

training_args = TrainingArguments(
    output_dir="./vit-shark-classifier2",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

from evaluate import load
accuracy = load("accuracy")

os.environ["WANDB_DISABLED"] = "true"

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    image_processor=feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:

from PIL import Image
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
from google.colab import files
import io

# تحميل الملف من الجهاز المحلي
uploaded = files.upload()
image_path = list(uploaded.keys())[0]  # أول ملف مرفوع

# تحميل النموذج (تأكد إنك محمل ومهيئ النموذج مسبقًا باسم model)
model.eval()

# ترتيب الأصناف
class_names = [
    'Blacktip_Shark', 'Bull_Shark',
    'Great_White_Shark', 'Hammerhead_Shark',
    'Mako_Shark', 'Tiger_Shark',
    'Whale_Shark', 'Whitetip_Shark'
]

# نفس التحويلات المستخدمة في التدريب
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# فتح الصورة
image = Image.open(image_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0)

# التوقع
with torch.no_grad():
    outputs = model(input_tensor)
    predicted_class = outputs.argmax(dim=1).item()

# عرض النتيجة
predicted_label = class_names[predicted_class]
print(f"✅ Predicted Shark Type: {predicted_label}")

# عرض الصورة
plt.imshow(image)
plt.title(f"Prediction: {predicted_label}")
plt.axis('off')
plt.show()

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files

# تفعيل GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تحميل النموذج وتحديد eval mode
model = torch.load("/content/model.pt", map_location=device)
model.eval()
model.to(device)

# الأصناف
class_names = [
    'Blacktip_Shark', 'Bull_Shark',
    'Great_White_Shark', 'Hammerhead_Shark',
    'Mako_Shark', 'Tiger_Shark',
    'Whale_Shark', 'Whitetip_Shark'
]

# التحويل
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# رفع الصورة
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
image = Image.open(image_path).convert("RGB")

# معالجة الصورة
input_tensor = transform(image).unsqueeze(0).to(device)

# التنبؤ
with torch.no_grad():
    output = model(input_tensor)
    pred_idx = output.argmax(1).item()
    label = class_names[pred_idx]

# عرض
print("✅ نوع القرش:", label)
plt.imshow(image)
plt.title(f"Prediction: {label}")
plt.axis('off')
plt.show()

In [12]:
dataset = ImageFolder("/content/Shark_project_split/train")
print(dataset.classes)

['Blacktip_Shark', 'Bull_Shark', 'Great_White_Shark', 'Hammerhead_Shark', 'Mako_Shark', 'Tiger_Shark', 'Whale_Shark', 'Whitetip_Shark']
